# Assignment 2 - Cascading Meta Learning
 
**Authors:**

1.   Liav Bachar 205888472
2.   Naor Kolet 205533060


# 0. Imports

In [5]:
import pandas as pd
import numpy as np
 
# TensorFlow
import tensorflow as tf
# from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
# from tensorflow.keras.layers import Input, BatchNormalization, Concatenate
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.applications import ResNet50V2
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Scikit-learn
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import log_loss
 
# Plots
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Misc.
from scipy.io import loadmat
import os
import random
import joblib
import cv2
from glob import glob
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Download Data

In [6]:
# Another dataset? https://www.tensorflow.org/tutorials/load_data/images
if not os.path.exists(r'./datasets/'):
    !mkdir ./datasets
    !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz' -P './datasets/'
    !tar -xf ./datasets/102flowers.tgz -C ./datasets/
    
#     !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102segmentations.tgz' -P './datasets/'
    
    !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat' -P './datasets/'
    
labels = loadmat('./datasets/imagelabels.mat')['labels'].reshape(-1)

In [33]:
def load_image(image_path):
#     img = cv2.imread(image_path)/255
    img = tf.io.read_file(image_path)
#     img = (cv2.resize(img, (105, 105)))
    print(img.shape)
    return img

In [4]:
images_path = glob('./datasets/jpg/*')

In [17]:
split_seed = 42
# 50% 25% 25%
train_paths, val_tst_paths, train_labels, val_tst_labels = train_test_split(images_path, labels, train_size=0.5, random_state=split_seed, shuffle=True, stratify=labels)
val_paths, tst_paths, val_labels, tst_labels = train_test_split(val_tst_paths, val_tst_labels, train_size=0.5, random_state=split_seed, shuffle=True, stratify=val_tst_labels)
len(train_paths),len(val_paths),len(tst_paths),

(4094, 2047, 2048)

In [28]:
# https://www.tensorflow.org/guide/data#decoding_image_data_and_resizing_it
train = tf.data.Dataset.list_files(train_paths)

In [36]:
load_image(next(iter(train)))

()


<tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01\xf4\x02\x11\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc